In [1]:
import codecs
import csv
import time
import os
import re
import gzip
import pandas as pd

In [2]:
PATH_TO_DATA = 'Data/'
PATH_TO_DATA_EN = PATH_TO_DATA+"enwiki/"
PATH_TO_DATA_UK = PATH_TO_DATA+"ukwiki/"

In [38]:
ENWIKI_ART_FNMS = []
for file in os.listdir(PATH_TO_DATA_EN):
    if re.match(r"enwiki-20180620-pages-meta-current\d{2}-p\d+p\d+.xml_art.csv.gz", file):
        ENWIKI_ART_FNMS.append(file)
ENWIKI_RED_FNMS = []
for file in os.listdir(PATH_TO_DATA_EN):
    if re.match(r"enwiki-20180620-pages-meta-current\d{2}-p\d+p\d+.xml_red.csv.gz", file):
        ENWIKI_RED_FNMS.append(file)        

In [4]:
def unpack(file_name):
    file_name_new = file_name.replace(".gz","")
    with gzip.open(file_name, 'rb') as f_in, open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    return file_name_new

In [5]:
def pack_and_remove(file_name):
    file_name_new = file_name+'.gz'
    with open(file_name, 'rb') as f_in, gzip.open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    os.remove(file_name)
    return file_name_new

In [31]:
df_id_name = None
for fn in ENWIKI_ART_FNMS:
    fn = PATH_TO_DATA_EN+fn
    print(fn)
    fn_new = unpack(fn)
    df_articles = pd.read_csv(fn_new, encoding='ISO-8859-1', quotechar="\"")
    df_id_name_tmp = df_articles[['id', 'title']].drop_duplicates()
    #print("df_id_name_tmp size: {}".format(df_id_name_tmp.shape))
    if df_id_name is not None:
        df_id_name = df_id_name.append(df_id_name_tmp)        
        #print("append")
    else:
        df_id_name = df_id_name_tmp
        #print("assign")
    print("df_id_name size: {}".format(df_id_name.shape))
    os.remove(fn_new)

Data/enwiki/enwiki-20180620-pages-meta-current01-p10p30303.xml_art.csv.gz


c:\users\mgontar.artfulbits\appdata\local\continuum\anaconda3\envs\p2tw\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


df_id_name.size size: 30096
Data/enwiki/enwiki-20180620-pages-meta-current02-p30304p88444.xml_art.csv.gz
df_id_name.size size: 83772
Data/enwiki/enwiki-20180620-pages-meta-current03-p88445p200507.xml_art.csv.gz
df_id_name.size size: 214820
Data/enwiki/enwiki-20180620-pages-meta-current04-p200511p352689.xml_art.csv.gz
df_id_name.size size: 313050
Data/enwiki/enwiki-20180620-pages-meta-current05-p352690p565313.xml_art.csv.gz
df_id_name.size size: 443036
Data/enwiki/enwiki-20180620-pages-meta-current06-p565314p892912.xml_art.csv.gz
df_id_name.size size: 599436
Data/enwiki/enwiki-20180620-pages-meta-current07-p892914p1268691.xml_art.csv.gz
df_id_name.size size: 781112
Data/enwiki/enwiki-20180620-pages-meta-current08-p1268692p1791079.xml_art.csv.gz
df_id_name.size size: 991336
Data/enwiki/enwiki-20180620-pages-meta-current09-p1791080p2336422.xml_art.csv.gz
df_id_name.size size: 1204988
Data/enwiki/enwiki-20180620-pages-meta-current10-p2336425p3046511.xml_art.csv.gz
df_id_name.size size: 145

In [34]:
df_id_name = df_id_name.drop_duplicates()

In [35]:
df_id_name.to_csv(PATH_TO_DATA_EN+'enwiki-20180620-id_name.csv', index = False, encoding='ISO-8859-1', quotechar="'", escapechar ="\\", quoting=csv.QUOTE_NONNUMERIC)

In [75]:
df_id_name = pd.read_csv(PATH_TO_DATA_EN+'enwiki-20180620-id_name.csv', encoding='ISO-8859-1', quotechar="'", escapechar ="\\")

In [76]:
for fn in ENWIKI_ART_FNMS:
    fn = PATH_TO_DATA_EN+fn
    print(fn)
    fn_new = unpack(fn)
    df_articles = pd.read_csv(fn_new, encoding='ISO-8859-1', quotechar="\"")
    df_articles = pd.merge(df_articles, df_id_name, how='left', left_on=['link_val'], right_on=['title'])
    df_articles = df_articles[['id_x', 'id_y', 'link_pos', 'link_txt']]
    df_articles.columns = ['id', 'link_id', 'link_pos', 'link_text']
    df_articles[['link_id']] = df_articles[['link_id']].fillna(-1).astype(int)
    df_articles[['link_pos']] = df_articles[['link_pos']].fillna(-1).astype(int)
    df_articles['is_red_link'] = False
    df_articles[df_articles[['link_id']] == -1]['is_red_link'] = True
    df_articles.to_csv(fn_new, index = False, encoding='ISO-8859-1', quotechar="'", 
                       escapechar ="\\", quoting=csv.QUOTE_NONNUMERIC)
    pack_and_remove(fn_new)

Data/enwiki/enwiki-20180620-pages-meta-current01-p10p30303.xml_art.csv.gz


c:\users\mgontar.artfulbits\appdata\local\continuum\anaconda3\envs\p2tw\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Data/enwiki/enwiki-20180620-pages-meta-current02-p30304p88444.xml_art.csv.gz
Data/enwiki/enwiki-20180620-pages-meta-current03-p88445p200507.xml_art.csv.gz
Data/enwiki/enwiki-20180620-pages-meta-current04-p200511p352689.xml_art.csv.gz
Data/enwiki/enwiki-20180620-pages-meta-current05-p352690p565313.xml_art.csv.gz
Data/enwiki/enwiki-20180620-pages-meta-current06-p565314p892912.xml_art.csv.gz
Data/enwiki/enwiki-20180620-pages-meta-current07-p892914p1268691.xml_art.csv.gz
Data/enwiki/enwiki-20180620-pages-meta-current08-p1268692p1791079.xml_art.csv.gz
Data/enwiki/enwiki-20180620-pages-meta-current09-p1791080p2336422.xml_art.csv.gz
Data/enwiki/enwiki-20180620-pages-meta-current10-p2336425p3046511.xml_art.csv.gz
Data/enwiki/enwiki-20180620-pages-meta-current11-p3046514p3926861.xml_art.csv.gz
Data/enwiki/enwiki-20180620-pages-meta-current12-p3926863p5040436.xml_art.csv.gz
Data/enwiki/enwiki-20180620-pages-meta-current13-p5040438p6197594.xml_art.csv.gz
Data/enwiki/enwiki-20180620-pages-meta-cur

In [88]:
UKWIKI_ART_FNMS = []
for file in os.listdir(PATH_TO_DATA_UK):
    if re.match(r"ukwiki-20180620-pages-meta-current\d{2}-p\d+p\d+.xml_art.csv.gz", file):
        UKWIKI_ART_FNMS.append(file)
UKWIKI_RED_FNMS = []
for file in os.listdir(PATH_TO_DATA_UK):
    if re.match(r"ukwiki-20180620-pages-meta-current\d{2}-p\d+p\d+.xml_red.csv.gz", file):
        UKWIKI_RED_FNMS.append(file)   

In [89]:
df_id_name = None
for fn in UKWIKI_ART_FNMS:
    fn = PATH_TO_DATA_UK+fn
    print(fn)
    fn_new = unpack(fn)
    df_articles = pd.read_csv(fn_new, encoding='UTF-8', quotechar="\"")
    df_id_name_tmp = df_articles[['id', 'title']].drop_duplicates()
    #print("df_id_name_tmp size: {}".format(df_id_name_tmp.shape))
    if df_id_name is not None:
        df_id_name = df_id_name.append(df_id_name_tmp)        
        #print("append")
    else:
        df_id_name = df_id_name_tmp
        #print("assign")
    print("df_id_name size: {}".format(df_id_name.shape))
    os.remove(fn_new)
df_id_name = df_id_name.drop_duplicates()
df_id_name.to_csv(PATH_TO_DATA_UK+'ukwiki-20180620-id_name.csv', index = False, encoding='UTF-8', quotechar="\"")

Data/ukwiki/ukwiki-20180620-pages-meta-current01-p1p5501930.xml_art.csv.gz


c:\users\mgontar.artfulbits\appdata\local\continuum\anaconda3\envs\p2tw\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


df_id_name size: (195092, 2)
Data/ukwiki/ukwiki-20180620-pages-meta-current02-p5501931p11003859.xml_art.csv.gz
df_id_name size: (426151, 2)
Data/ukwiki/ukwiki-20180620-pages-meta-current03-p11003860p16505788.xml_art.csv.gz
df_id_name size: (639780, 2)
Data/ukwiki/ukwiki-20180620-pages-meta-current04-p16505789p22007717.xml_art.csv.gz
df_id_name size: (796717, 2)


In [ ]:
for fn in UKWIKI_ART_FNMS:
    fn = PATH_TO_DATA_UK+fn
    print(fn)
    fn_new = unpack(fn)
    df_articles = pd.read_csv(fn_new, encoding='UTF-8', quotechar="\"")
    df_articles = pd.merge(df_articles, df_id_name, how='left', left_on=['link_val'], right_on=['title'])
    df_articles = df_articles[['id_x', 'id_y', 'link_pos', 'link_txt']]
    df_articles.columns = ['id', 'link_id', 'link_pos', 'link_text']
    df_articles[['link_id']] = df_articles[['link_id']].fillna(-1).astype(int)
    df_articles[['link_pos']] = df_articles[['link_pos']].fillna(-1).astype(int)
    df_articles['is_red_link'] = False
    df_articles[df_articles[['link_id']] == -1]['is_red_link'] = True
    df_articles.to_csv(fn_new, index = False, encoding='UTF-8', quotechar="\"")
    pack_and_remove(fn_new)

In [90]:
df_id_name = pd.read_csv(PATH_TO_DATA_EN+'enwiki-20180620-id_name.csv', encoding='ISO-8859-1', quotechar="'", escapechar ="\\")

In [95]:
fn = PATH_TO_DATA_UK+'ukwiki-20180620-langlinks_en.csv.gz'
print(fn)
fn_new = unpack(fn)
df_uk_en_links = pd.read_csv(fn_new, encoding='ISO-8859-1', quotechar="'")

df_uk_en_links = pd.merge(df_uk_en_links, df_id_name, how='left', left_on=['ll_title'], right_on=['title'])
df_uk_en_links

Data/ukwiki/ukwiki-20180620-langlinks_en.csv.gz


,ll_from,ll_lang,ll_title,id,title
0,3,en,NaN,49244.0,NaN
1,93215,en,NaN,49244.0,NaN
2,109484,en,NaN,49244.0,NaN
3,122051,en,NaN,49244.0,NaN
4,130506,en,NaN,49244.0,NaN
5,609236,en,NaN,49244.0,NaN
6,621969,en,NaN,49244.0,NaN
7,772924,en,NaN,49244.0,NaN
8,907757,en,NaN,49244.0,NaN
9,942309,en,NaN,49244.0,NaN


In [96]:
df_uk_en_links = df_uk_en_links[['ll_from', 'id']]
df_uk_en_links.columns = ['id_uk', 'id_en']
df_uk_en_links[['id_uk']] = df_uk_en_links[['id_uk']].fillna(-1).astype(int)
df_uk_en_links[['id_en']] = df_uk_en_links[['id_en']].fillna(-1).astype(int)
df_uk_en_links.to_csv(PATH_TO_DATA_UK+'20180620-langlinks_uk_en.csv', index = False, encoding='ISO-8859-1', quotechar="'", 
                       escapechar ="\\", quoting=csv.QUOTE_NONNUMERIC)
pack_and_remove(fn_new)

c:\users\mgontar.artfulbits\appdata\local\continuum\anaconda3\envs\p2tw\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


'Data/ukwiki/ukwiki-20180620-langlinks_en.csv.gz'